In [198]:
import pandas as pd
import numpy as np
import seaborn as sb
import io
import os

# Data Engineering

### Load in data, drop NA's for additional variables, and set column types

In [199]:
os.getcwd()

'/Users/yazeedabulata/PycharmProjects/group-project-project-group-6/data'

In [200]:
data = pd.read_csv('organisations.csv')
fdi = pd.read_csv('fdi_usd.csv', sep= ';')
gdp = pd.read_csv('gdp_per_capita_usd.csv', sep=';')

In [201]:
gdp=gdp.dropna(how='any')
fdi=fdi.dropna(how='any')

In [202]:
data = data.drop(['permalink', 'homepage_url'], axis=1)

In [203]:
data[['market','status','country_code','state_code','region','city']] = data[['market','status','country_code','state_code','region','city']].astype('category')
data[['funding_total_usd']] = data[['funding_total_usd']].astype('int64')

In [204]:
df_final = data
df_final

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,#waywire,|Entertainment|Politics|Social Media|News|,News,1750000,acquired,USA,NY,New York City,New York,1,2012-06-01 00:00:00,2012-06-30 00:00:00,2012-06-30 00:00:00
1,'Rock' Your Paper,|Publishing|Education|,Publishing,40000,operating,EST,NaN,Tallinn,Tallinn,1,2012-10-26 00:00:00,2012-08-09 00:00:00,2012-08-09 00:00:00
2,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000,operating,GBR,NaN,London,London,1,2011-04-01 00:00:00,2011-04-01 00:00:00,2011-04-01 00:00:00
3,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,60000,operating,USA,TX,Dallas,Fort Worth,2,2014-01-01 00:00:00,2014-08-17 00:00:00,2014-09-26 00:00:00
4,.Club Domains,|Software|,Software,7000000,NaN,USA,FL,Ft. Lauderdale,Oakland Park,1,2011-10-10 00:00:00,2013-05-31 00:00:00,2013-05-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28724,Zyraz Technology,|Biotechnology|,Biotechnology,15419877,closed,MYS,NaN,MYS - Other,Damansara New Village,4,1994-01-01 00:00:00,2008-01-01 00:00:00,2013-02-15 00:00:00
28725,Zytoprotec,|Biotechnology|,Biotechnology,2686600,operating,AUT,NaN,Vienna,Gerasdorf Bei Wien,1,2007-01-01 00:00:00,2013-01-29 00:00:00,2013-01-29 00:00:00
28726,Zzish,|Analytics|Gamification|Developer APIs|iOS|And...,Education,320000,operating,GBR,NaN,London,London,1,2013-01-28 00:00:00,2014-03-24 00:00:00,2014-03-24 00:00:00
28727,Zzzzapp Wireless ltd.,|Web Development|Advertising|Wireless|Mobile|,Web Development,97398,operating,HRV,NaN,Split,Split,5,2012-05-13 00:00:00,2011-11-01 00:00:00,2014-09-10 00:00:00


### Adding Effective date/time columns

In [205]:
#DateTime Columns
df_final['founded_at']=pd.to_datetime(df_final['founded_at'], errors='coerce')
df_final['first_funding_at']=pd.to_datetime(df_final['first_funding_at'], errors='coerce')
df_final['last_funding_at']=pd.to_datetime(df_final['last_funding_at'], errors='coerce')
#Years
df_final['founding_year']= df_final['founded_at'].dt.year
df_final['founding_month']= df_final['founded_at'].dt.month
df_final['founding_day']= df_final['founded_at'].dt.day
#Months
df_final['first_funding_year']= df_final['first_funding_at'].dt.year
df_final['first_funding_month']= df_final['first_funding_at'].dt.month
df_final['first_funding_day']= df_final['first_funding_at'].dt.day
#Days
df_final['last_funding_year']= df_final['last_funding_at'].dt.year
df_final['last_funding_month']= df_final['last_funding_at'].dt.month
df_final['last_funding_day']= df_final['last_funding_at'].dt.day
#Funding Period
df_final['funding_period_days'] = (df_final['last_funding_at'] - df_final['first_funding_at']).dt.days
df_final['t_until_first_funding'] = (df_final['first_funding_at'] - df_final['founded_at']).dt.days
df_final['t_until_last_funding'] = (df_final['last_funding_at'] - df_final['founded_at']).dt.days

def timing(df_final):
    if (df_final['t_until_first_funding'] < 0):
        return 'fundedbeforefounding'
    elif (df_final['t_until_first_funding'] == 0):
        return 'fundedwithfounding'
    else:
        return 'fundedafterfounding'
df_final['fund_time'] = df_final.apply(timing, axis=1)

#df_final['fund_time'] = (df_final['t_until_first_funding']<=0).astype('int64')
df_final=df_final.drop(columns=['founded_at', 'first_funding_at', 'last_funding_at'])
df_final

,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founding_year,founding_month,founding_day,first_funding_year,first_funding_month,first_funding_day,last_funding_year,last_funding_month,last_funding_day,funding_period_days,t_until_first_funding,t_until_last_funding,fund_time
0,#waywire,|Entertainment|Politics|Social Media|News|,News,1750000,acquired,USA,NY,New York City,New York,1,2012,6,1,2012,6,30,2012,6,30,0,29,29,fundedafterfounding
1,'Rock' Your Paper,|Publishing|Education|,Publishing,40000,operating,EST,NaN,Tallinn,Tallinn,1,2012,10,26,2012,8,9,2012,8,9,0,-78,-78,fundedbeforefounding
2,(In)Touch Network,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,1500000,operating,GBR,NaN,London,London,1,2011,4,1,2011,4,1,2011,4,1,0,0,0,fundedwithfounding
3,-R- Ranch and Mine,|Tourism|Entertainment|Games|,Tourism,60000,operating,USA,TX,Dallas,Fort Worth,2,2014,1,1,2014,8,17,2014,9,26,40,228,268,fundedafterfounding
4,.Club Domains,|Software|,Software,7000000,NaN,USA,FL,Ft. Lauderdale,Oakland Park,1,2011,10,10,2013,5,31,2013,5,31,0,599,599,fundedafterfounding
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28724,Zyraz Technology,|Biotechnology|,Biotechnology,15419877,closed,MYS,NaN,MYS - Other,Damansara New Village,4,1994,1,1,2008,1,1,2013,2,15,1872,5113,6985,fundedafterfounding
28725,Zytoprotec,|Biotechnology|,Biotechnology,2686600,operating,AUT,NaN,Vienna,Gerasdorf Bei Wien,1,2007,1,1,2013,1,29,2013,1,29,0,2220,2220,fundedafterfounding
28726,Zzish,|Analytics|Gamification|Developer APIs|iOS|And...,Education,320000,operating,GBR,NaN,London,London,1,2013,1,28,2014,3,24,2014,3,24,0,420,420,fundedafterfounding
28727,Zzzzapp Wireless ltd.,|Web Development|Advertising|Wireless|Mobile|,Web Development,97398,operating,HRV,NaN,Split,Split,5,2012,5,13,2011,11,1,2014,9,10,1044,-194,850,fundedbeforefounding


### Dropping negligible variables

In [206]:
#deciding which variables to keep/remove and turning categoricals into dummy variables
#these variables were removed to. a logical deduction of their lack of relevance to the model:
df_final=df_final.drop(columns=['name', 'region','first_funding_year','first_funding_month','first_funding_day',
                               'last_funding_year', 'last_funding_month', 'last_funding_day','founding_day',])
#the only location variable used was country

#after testing models with the following variables and checking the "features importance" it was deduced that the categorical variables add too much confusion to the model. To combat this the variables were re-tested using only categories that feature at least 10 samples. The following variables still do not improve the model.

df_final=df_final.drop(columns=['state_code', 'status', 'category_list', 'city', 'market'])
df_final

,funding_total_usd,country_code,funding_rounds,founding_year,founding_month,funding_period_days,t_until_first_funding,t_until_last_funding,fund_time
0,1750000,USA,1,2012,6,0,29,29,fundedafterfounding
1,40000,EST,1,2012,10,0,-78,-78,fundedbeforefounding
2,1500000,GBR,1,2011,4,0,0,0,fundedwithfounding
3,60000,USA,2,2014,1,40,228,268,fundedafterfounding
4,7000000,USA,1,2011,10,0,599,599,fundedafterfounding
...,...,...,...,...,...,...,...,...,...
28724,15419877,MYS,4,1994,1,1872,5113,6985,fundedafterfounding
28725,2686600,AUT,1,2007,1,0,2220,2220,fundedafterfounding
28726,320000,GBR,1,2013,1,0,420,420,fundedafterfounding
28727,97398,HRV,5,2012,5,1044,-194,850,fundedbeforefounding


### Splitting categorical variables into into onehot matrix

In [207]:
#Market and status were cancelled out as the variable didn't add to the model performance

#market = pd.get_dummies(df_final['market'])
#df_final = pd.concat([df_final, market], axis=1)
#df_final.drop('market',axis=1,inplace=True)
#df_final = df_final.iloc[:, :-1] #done because column label wouldn't work
#status = pd.get_dummies(df_final['status'])
#df_final = pd.concat([df_final, status], axis=1)
#df_final.drop('status',axis=1,inplace=True)
#df_final.drop('closed',axis=1,inplace=True)

country_code = pd.get_dummies(df_final['country_code'])
df_final = pd.concat([df_final, country_code], axis=1)
df_final.drop('country_code',axis=1,inplace=True)
df_final.drop('SWE',axis=1,inplace=True)

fund_time = pd.get_dummies(df_final['fund_time'])
df_final = pd.concat([df_final, fund_time], axis=1)
df_final.drop('fund_time',axis=1,inplace=True)
df_final.drop('fundedafterfounding',axis=1,inplace=True)


pd.set_option('display.max_columns', None)
df_final

,funding_total_usd,funding_rounds,founding_year,founding_month,funding_period_days,t_until_first_funding,t_until_last_funding,ARE,ARG,ARM,AUS,AUT,AZE,BEL,BGD,BGR,BHR,BHS,BLR,BMU,BRA,BWA,CAN,CHE,CHL,CHN,CMR,COL,CRI,CYM,CYP,CZE,DEU,DNK,DOM,DZA,ECU,EGY,ESP,EST,FIN,FRA,GBR,GHA,GIB,GRC,GTM,HKG,HRV,HUN,IDN,IND,IRL,ISL,ISR,ITA,JAM,JOR,JPN,KEN,KHM,KOR,LBN,LTU,LUX,LVA,MAF,MAR,MDA,MEX,MKD,MLT,MMR,MUS,MYS,NGA,NLD,NOR,NPL,NZL,OMN,PAK,PAN,PER,PHL,POL,PRT,ROM,RUS,SAU,SGP,SLV,SOM,SRB,SVK,SVN,SYC,THA,TTO,TUN,TUR,TWN,TZA,UGA,UKR,URY,USA,UZB,VNM,ZAF,fundedbeforefounding,fundedwithfounding
0,1750000,1,2012,6,0,29,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,40000,1,2012,10,0,-78,-78,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1500000,1,2011,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,60000,2,2014,1,40,228,268,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,7000000,1,2011,10,0,599,599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28724,15419877,4,1994,1,1872,5113,6985,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28725,2686600,1,2007,1,0,2220,2220,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28726,320000,1,2013,1,0,420,420,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
28727,97398,5,2012,5,1044,-194,850,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# Model Creation

### Creating Train Test Split

In [208]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_final,train_size=0.7, random_state=1)

In [209]:
train_X = df_train.drop('funding_total_usd',axis=1)
train_y = df_train['funding_total_usd']
test_X = df_test.drop('funding_total_usd',axis=1)
test_y = df_test['funding_total_usd']
train_y

22372    34000000
14497      550000
1419     42000000
3909       250000
12674     2250000
           ...   
10955       75000
17289      585000
5192      1550000
12172      300000
235       1380700
Name: funding_total_usd, Length: 20110, dtype: int64

### Creating and Testing Model

In [210]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(learning_rate=0.05, max_depth = 5, min_samples_leaf=60, n_estimators=100)
model.fit(train_X, train_y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=5, min_samples_leaf=60)

In [211]:
model.score(train_X,train_y)

0.14138525519994338

In [212]:
model.score(test_X,test_y)

0.16015594461273674

#This was the final grid search we ran to optimize the parameters -
from sklearn.model_selection import GridSearchCV

parameters = {'learning_rate': [0.01,0.05, 0.001],
                  'min_samples_leaf': [20,35,60],
                  'n_estimators' : [100, 200],
                  'max_depth'    : [5,10,15]
                 }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 5)

grid.fit(train_X,train_y)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
print("\n The best score across ALL searched params:\n",grid.best_score_)
print("\n The best parameters across ALL searched params:\n",grid.best_params_)

# Other tests and variables we tried

#### We tried using category as a dummy variable
#This ultimately lowered the effectiveness of our model

#Splitting Category List
df = data['category_list']
df = df.str.split('|', expand=True, )
ncategories = pd.concat([df[0],df[1],df[2],df[3],df[4],df[5],df[6],df[7],df[8],df[9],df[10],df[11],df[12],df[13],df[14],df[15], df[16],df[17],df[18],df[19],df[20],df[21]])
binary_categories = df.stack().str.get_dummies().sum(level=0)
binary_categories = binary_categories.apply(pd.to_numeric)
binary_categories
df_temp = pd.concat([data, binary_categories], axis=1)
binary_temp = df_temp.iloc[:, 13:805]
binary_temp = binary_temp.fillna(0).astype('int64')
binary_temp.isnull().sum()
df_final = pd.concat([data, binary_temp], axis=1)
df_final

### We tried adding FDI and GDP as additional variables
fdi['founding_year'] = fdi['year']
fdi = fdi.drop(columns=['year'])
df_with_fdi = pd.merge(df_final, fdi,  how='left', left_on=['country_code', 'founding_year'], right_on = ['country_code','founding_year'])
gdp['country_code']=gdp['Country Code']
gdp['founding_year']=gdp['year']
gdp = gdp.drop(columns=['year','Country Code'])
df_final = pd.merge(df_with_fdi, gdp,  how='left', left_on=['country_code', 'founding_year'], right_on = ['country_code','founding_year'])
df_final.drop('country_code',axis=1,inplace=True)

#### We tried logging them as well to normalize their effect
#Getting log values of GDP and FDI
df_final['FDI_log10'] = np.log2(df_final['total_fdi_in_usd'])
df_final.drop('total_fdi_in_usd',axis=1,inplace=True)
df_final['GDP_log10'] = np.log2(df_final['gdp_per_capita_usd'])
df_final.drop('gdp_per_capita_usd',axis=1,inplace=True)

### We tried using categorical variables that had a frequency greater than 30
#The first part of this code was located  in the first section
countryfreq = data.groupby('country_code').size()
countryfreq = countryfreq.to_frame()
cf_10 = countryfreq[countryfreq[0]>30]
countries = cf_10[0].index.values.tolist()

marketfreq = data.groupby('market').size()
marketfreq = marketfreq.to_frame()
mf_10 = marketfreq[marketfreq[0]>30]
markets = mf_10[0].index.values.tolist()

#The second part used these columns to only keep the desired ones in the model
numerical_columns = ['df_final.columns', 'funding_rounds', 'founding_year', 'founding_month', 'funding_period_days', 't_until_first_funding', 't_until_last_funding', 'fundedbeforefounding', 'fundedwithfounding', 'funding_total_usd']
wanted_columns = numerical_columns + countries + markets
wanted_columns

#wanted_columns = countries.append(numerical_columns)
#wanted_columns
df_final = df_final[df_final.columns.intersection(wanted_columns)]
#pandas_df

### We tried deleting variables that had no explanatory power but later realized that would have no effect

important = model.feature_importances_!=0
important
nonimportant = np.where(important==False)
LNI = nonimportant[0].tolist()
print(LNI)
LNI1 = [x + 1 for x in LNI]
LNI1
df_adjusted = df_final.drop(df_final.columns[LNI1], axis=1)
df_adjusted


#Rerun model without negligible variables
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_adjusted, train_size=0.7, random_state=1)
train_Xa = df_train.drop('funding_total_usd', axis=1)
train_ya = df_train['funding_total_usd']
test_Xa = df_test.drop('funding_total_usd', axis=1)
test_ya = df_test['funding_total_usd']
test_Xa

# Other models we tried

#### We tried fitting a polynomial regression
from sklearn.model_selection import GridSearchCV

parameters = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]
                 }

grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 5)

grid.fit(train_X,train_y)
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
print("\n The best score across ALL searched params:\n",grid.best_score_)
print("\n The best parameters across ALL searched params:\n",grid.best_params_)